# Load GISS-H output from Nick and recompute necessary grid information

To compute transports

For some reason, this seems to be interpolated to a different grid than the GISS-G dataset.
Why?

In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from xgcm import Grid
from pych.calc import haversine

In [2]:
giss_dir = '/workspace/results/giss-euc/ocn_1992-2019_h/'

In [3]:
giss = xr.open_dataset(f'{giss_dir}/1992-2019.uh.nc')

for f in ['v','t','s']:
    fld = f if f[-1]!='_' else f[:-1]
    giss[fld] = xr.open_dataarray(f'{giss_dir}/1992-2019.{f}h.nc')

In [4]:
giss

<xarray.Dataset>
Dimensions:  (lat: 17, lon: 360, record: 336, z: 17)
Coordinates:
  * lat      (lat) float32 -3.5 -2.5 -1.5 -0.5 0.5 ... 8.5 9.5 10.5 11.5 12.5
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * z        (z) float32 0.0 10.0 20.0 30.0 50.0 ... 500.0 600.0 700.0 800.0
Dimensions without coordinates: record
Data variables:
    U        (record, z, lat, lon) float32 ...
    v        (record, z, lat, lon) float32 ...
    t        (record, z, lat, lon) float32 ...
    s        (record, z, lat, lon) float32 ...
Attributes:
    title:                     RunId Mean a-e Eh213f10 year=1992 month=JAN
    history:                   Fri Apr 10 16:08:19 2020: ncecat 1992-01.uh.nc...
    nco_openmp_thread_number:  1
    NCO:                       4.2.3

In [5]:
time = np.arange('1992-01','2020-01',dtype='datetime64')

In [6]:
assert len(time)==len(giss.record)

In [7]:
giss = giss.rename({'U':'u'})

### Units

GISS saves U and V in cm/s, swap to m/s

In [8]:
giss.u

<xarray.DataArray 'u' (record: 336, z: 17, lat: 17, lon: 360)>
[34957440 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -3.5 -2.5 -1.5 -0.5 0.5 ... 8.5 9.5 10.5 11.5 12.5
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * z        (z) float32 0.0 10.0 20.0 30.0 50.0 ... 500.0 600.0 700.0 800.0
Dimensions without coordinates: record
Attributes:
    units:      cm/s
    long_name:  eastward velocity

In [9]:
for f in ['u','v']:
    att = giss[f].attrs
    giss[f]=giss[f]/100
    giss[f].attrs=att
    giss[f].attrs['units']='m/s'

In [10]:
giss['time']=xr.DataArray(time,coords=giss.record.coords,dims='record')

In [11]:
giss = giss.swap_dims({'record':'time'})

In [12]:
giss

<xarray.Dataset>
Dimensions:  (lat: 17, lon: 360, time: 336, z: 17)
Coordinates:
  * lat      (lat) float32 -3.5 -2.5 -1.5 -0.5 0.5 ... 8.5 9.5 10.5 11.5 12.5
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * z        (z) float32 0.0 10.0 20.0 30.0 50.0 ... 500.0 600.0 700.0 800.0
  * time     (time) datetime64[ns] 1992-01-01 1992-02-01 ... 2019-12-01
Data variables:
    u        (time, z, lat, lon) float32 -0.07827317 ... -0.0036864323
    v        (time, z, lat, lon) float32 -0.049138803 ... -0.002400057
    t        (time, z, lat, lon) float32 ...
    s        (time, z, lat, lon) float32 ...
Attributes:
    title:                     RunId Mean a-e Eh213f10 year=1992 month=JAN
    history:                   Fri Apr 10 16:08:19 2020: ncecat 1992-01.uh.nc...
    nco_openmp_thread_number:  1
    NCO:                       4.2.3

### Rename some coordinates

Just use MITgcm style to make it easy for me

the atmospheric variables sit at the intersection of four grid cells, hence why the XG/YG points align with the atm variable lon/lat

In [13]:
giss = giss.rename({'lon':'XC','lat':'YC','z':'Zl'})

In [14]:
giss.data_vars

Data variables:
    u        (time, Zl, YC, XC) float32 -0.07827317 ... -0.0036864323
    v        (time, Zl, YC, XC) float32 -0.049138803 ... -0.002400057
    t        (time, Zl, YC, XC) float32 ...
    s        (time, Zl, YC, XC) float32 ...

In [15]:
giss.YC

<xarray.DataArray 'YC' (YC: 17)>
array([-3.5, -2.5, -1.5, -0.5,  0.5,  1.5,  2.5,  3.5,  4.5,  5.5,  6.5,  7.5,
        8.5,  9.5, 10.5, 11.5, 12.5], dtype=float32)
Coordinates:
  * YC       (YC) float32 -3.5 -2.5 -1.5 -0.5 0.5 1.5 ... 8.5 9.5 10.5 11.5 12.5
Attributes:
    units:      degrees_north
    long_name:  latitude

### Add some coordinates to make this the same as the other dataset

Compare to what we have for GISS-G

In [16]:
gg = xr.open_dataset('/workspace/results/giss-euc/giss_g.nc')

In [17]:
gg.YG

<xarray.DataArray 'YG' (YG: 16)>
array([-3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.,
       11., 12.], dtype=float32)
Coordinates:
  * YG       (YG) float32 -3.0 -2.0 -1.0 0.0 1.0 2.0 ... 8.0 9.0 10.0 11.0 12.0
Attributes:
    units:              degrees_north
    axis:               Y
    c_grid_axis_shift:  -0.5

In [18]:
giss['YG'] = xr.DataArray(giss.YC.values[:-1]+.5,coords={'YG':giss.YC.values[:-1]+.5},dims=('YG',))

Here the longitude coordinate is much different. 
Rather than a spacing of 1.25degrees, here it is 1 degree.

In [19]:
giss['XG'] = xr.DataArray(giss.XC.values-.5,coords={'XG':giss.XC.values-.5},dims=('XG',))

### Now there is one extra eastward velocity point which is giving some trouble

Weird, not sure why that is there...

In [20]:
giss = giss.drop_sel(YC=-3.5)

### Some meta data for xgcm

and another vertical coordinate...

In [21]:
giss.XC.attrs['axis'] = 'X'
giss.XG.attrs['axis'] = 'X'

giss.YC.attrs['axis'] = 'Y'
giss.YG.attrs['axis'] = 'Y'

giss.XG.attrs['c_grid_axis_shift']=-0.5
giss.YG.attrs['c_grid_axis_shift']=-0.5

## Vertical coordinate is different

Here I have to infer that this is the height of the "upper grid cell face" for each grid cell,
rather than the cell center, since it starts at zero.

In [22]:
giss.Zl

<xarray.DataArray 'Zl' (Zl: 17)>
array([  0.,  10.,  20.,  30.,  50.,  75., 100., 125., 150., 200., 250., 300.,
       400., 500., 600., 700., 800.], dtype=float32)
Coordinates:
  * Zl       (Zl) float32 0.0 10.0 20.0 30.0 50.0 ... 500.0 600.0 700.0 800.0
Attributes:
    units:      m
    long_name:  depth

Fake it, and extrapolate to get "other side"

In [23]:
dz = giss.Zl[-1]-giss.Zl[-2]
giss['Zu'] = np.concatenate([giss.Zl[1:],[(giss.Zl[-1]+dz)]])

In [24]:
giss.Zu

<xarray.DataArray 'Zu' (Zu: 17)>
array([ 10.,  20.,  30.,  50.,  75., 100., 125., 150., 200., 250., 300., 400.,
       500., 600., 700., 800., 900.], dtype=float32)
Coordinates:
  * Zu       (Zu) float32 10.0 20.0 30.0 50.0 75.0 ... 600.0 700.0 800.0 900.0

In [25]:
giss['Z'] = (giss.Zu.values-giss.Zl.values)/2 + giss.Zl.values

In [26]:
giss['Zp1'] = np.concatenate([[giss.Zl[0]],giss.Zu])

In [27]:
giss.Z.attrs['axis'] = 'Z'
giss.Zl.attrs['axis'] = 'Z'
giss.Zu.attrs['axis'] = 'Z'
giss.Zp1.attrs['axis'] = 'Z'

giss.XG.attrs['c_grid_axis_shift']=-0.5
giss.YG.attrs['c_grid_axis_shift']=-0.5
giss.Zl.attrs['c_grid_axis_shift']=-0.5
giss.Zu.attrs['c_grid_axis_shift']= 0.5
giss.Zp1.attrs['c_grid_axis_shift']=0.5

### Finally, get differential elements for computing transports

dx, dy...

In [28]:
xg,yg = np.meshgrid(giss.XG,giss.YG)

In [29]:
dxG = haversine(xg,yg,np.roll(xg,-1,axis=1),np.roll(yg,-1,axis=1))*1000

Since latitude not periodic, have to add on the boundary condition

In [30]:
xg_rolled = np.roll(xg,-1,axis=0)
yg_rolled = np.roll(yg,-1,axis=0)

yg_rolled[-1,:] = 13.*np.ones_like(xg_rolled[0,:])

In [31]:
dyG = haversine(xg_rolled,yg_rolled,xg,yg) * 1000

In [32]:
xc,yc = np.meshgrid(giss.XC,giss.YC)

In [33]:
dxC = haversine(xc,yc,np.roll(xc,-1,axis=1),np.roll(yc,-1,axis=1))*1000

In [34]:
xc_rolled = np.roll(xc,-1,axis=0)
yc_rolled = np.roll(yc,-1,axis=0)

yc_rolled[-1,:]=13.5*np.ones_like(yc_rolled[0,:])

In [35]:
dyC = haversine(xc_rolled,yc_rolled,xc,yc)*1000

In [36]:
giss['dxG'] = xr.DataArray(dxG,coords={'YG':giss.YG,'XC':giss.XC},dims=('YG','XC'),
                          attrs={'units':'m'})
giss['dyG'] = xr.DataArray(dyG,coords={'YC':giss.YC,'XG':giss.XG},dims=('YC','XG'),
                          attrs={'units':'m'})
giss['dxC'] = xr.DataArray(dxC,coords={'YC':giss.YC,'XG':giss.XG},dims=('YC','XG'),
                          attrs={'units':'m'})
giss['dyC'] = xr.DataArray(dyC,coords={'YG':giss.YG,'XC':giss.XC},dims=('YG','XC'),
                          attrs={'units':'m'})

In [37]:
for f in ['dxG','dxC','dyC','dyG']:
    giss=giss.set_coords(f)

### Now make a grid object

In [38]:
grid = Grid(giss,periodic='X')

In [39]:
grid

<xgcm.Grid>
Z Axis (not periodic):
  * center   Z --> left
  * left     Zl --> center
  * outer    Zp1 --> center
  * right    Zu --> center
X Axis (periodic):
  * center   XC --> left
  * left     XG --> center
Y Axis (not periodic):
  * center   YC --> left
  * left     YG --> center

In [40]:
giss['drF'] = grid.diff(giss.Zp1,'Z',to='center')

In [41]:
giss['dyF'] = grid.interp(giss.dyG,'X',boundary='fill')

In [42]:
giss['dxF'] = grid.interp(giss.dxG,'Y',boundary='fill')

### Reset variables to be associated with Z not Zl

In [44]:
for fld in giss.data_vars.keys():
    if 'Zl' in giss[fld].dims:
        giss[fld] = giss[fld].swap_dims({'Zl':'Z'})

In [46]:
giss = giss.set_coords(['drF','dxF','dyF'])

In [47]:
giss

<xarray.Dataset>
Dimensions:  (XC: 360, XG: 360, YC: 16, YG: 16, Z: 17, Zl: 17, Zp1: 18, Zu: 17, time: 336)
Coordinates:
  * Z        (Z) float64 5.0 15.0 25.0 40.0 62.5 ... 550.0 650.0 750.0 850.0
  * YC       (YC) float32 -2.5 -1.5 -0.5 0.5 1.5 2.5 ... 8.5 9.5 10.5 11.5 12.5
  * XC       (XC) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * Zl       (Zl) float32 0.0 10.0 20.0 30.0 50.0 ... 500.0 600.0 700.0 800.0
  * time     (time) datetime64[ns] 1992-01-01 1992-02-01 ... 2019-12-01
  * YG       (YG) float32 -3.0 -2.0 -1.0 0.0 1.0 2.0 ... 8.0 9.0 10.0 11.0 12.0
  * XG       (XG) float32 -180.0 -179.0 -178.0 -177.0 ... 177.0 178.0 179.0
  * Zu       (Zu) float32 10.0 20.0 30.0 50.0 75.0 ... 600.0 700.0 800.0 900.0
  * Zp1      (Zp1) float32 0.0 10.0 20.0 30.0 50.0 ... 600.0 700.0 800.0 900.0
    dxG      (YG, XC) float32 111043.42 111041.89 ... 108764.37 108766.26
    dyG      (YC, XG) float32 111194.94 111194.94 ... 111194.87 111194.87
    dxC      (YC, XG) float32 111088.47 111089.99 ... 108558.484 108560.37
    dyC      (YG, XC) float32 111194.914 111194.914 ... 111194.87 111194.87
    drF      (Z) float32 10.0 10.0 10.0 20.0 25.0 ... 100.0 100.0 100.0 100.0
    dyF      (YC, XC) float32 111194.94 111194.94 ... 111194.87 111194.87
    dxF      (YC, XC) float32 111085.75 111084.23 ... 54382.184 54383.13
Data variables:
    u        (time, Z, YC, XC) float32 -0.16269255 -0.15507309 ... -0.0036864323
    v        (time, Z, YC, XC) float32 -0.09098051 -0.10466799 ... -0.002400057
    t        (time, Z, YC, XC) float32 ...
    s        (time, Z, YC, XC) float32 ...
Attributes:
    title:                     RunId Mean a-e Eh213f10 year=1992 month=JAN
    history:                   Fri Apr 10 16:08:19 2020: ncecat 1992-01.uh.nc...
    nco_openmp_thread_number:  1
    NCO:                       4.2.3

### Interpolate velocities to where they normally would be on a C grid

In [45]:
#giss['u'] = grid.interp(giss.u,'X',boundary='fill')

In [46]:
#giss['v'] = grid.interp(giss.v,'Y',boundary='fill')

## Save this for later

In [48]:
giss.to_netcdf('/workspace/results/giss-euc/giss_h.nc')